<a href="https://colab.research.google.com/github/culiacanai/Aprende_Python_con_GoogleColab/blob/main/notebooks/09_Web_Scraping_Basico.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🕸️ Web Scraping Básico

### Aprende Python con Google Colab — por [Culiacan.AI](https://culiacan.ai)

**Nivel:** 🟡 Intermedio  
**Duración estimada:** 60 minutos  
**Requisitos:** Haber completado el [Notebook 07 — Pandas Básico](07_Pandas_Basico.ipynb)

---

En este notebook vas a:
- Entender qué es web scraping y cuándo usarlo (y cuándo no)
- Hacer peticiones HTTP con la librería `requests`
- Parsear HTML con `BeautifulSoup`
- Extraer texto, enlaces, tablas e imágenes de páginas web
- Limpiar y estructurar datos extraídos con Pandas
- Conocer las buenas prácticas y consideraciones éticas

> 💡 **Web scraping** es el proceso de extraer datos de páginas web de forma automática. Es una habilidad esencial para obtener datos que no están disponibles en APIs o archivos descargables.


---

## 0. Preparación


In [ ]:
# Instalar/importar librerías
!pip install requests beautifulsoup4 lxml -q

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

print("✅ Librerías listas")

---

## 1. ¿Qué es web scraping?

Imagina que quieres una tabla de datos de una página web. Podrías copiar y pegar manualmente... o podrías escribir un programa que lo haga por ti en segundos.

### El proceso básico:

```
1. Enviar petición HTTP → Obtener el HTML de la página
2. Parsear el HTML → Convertirlo en una estructura navegable
3. Extraer datos → Buscar los elementos que necesitas
4. Limpiar y guardar → Estructurar los datos en CSV, Excel, etc.
```

### ⚠️ Consideraciones éticas y legales

Antes de hacer scraping, siempre verifica:

| ✅ Está bien | ❌ Evita |
|-------------|----------|
| Datos públicos para uso personal/educativo | Datos protegidos por login |
| Respetar el archivo `robots.txt` | Hacer miles de peticiones rápidas |
| Citar la fuente de los datos | Revender datos extraídos |
| Usar APIs cuando estén disponibles | Ignorar los términos de servicio |

> 💡 **Regla de oro:** Si el sitio tiene una API, úsala en vez de hacer scraping. Es más confiable y respetuoso.


---

## 2. Hacer peticiones HTTP con requests

`requests` es la librería más popular de Python para hacer peticiones web.

### 2.1 Tu primera petición


In [ ]:
# Hacer una petición GET a una página
url = "https://quotes.toscrape.com/"
respuesta = requests.get(url)

# Información de la respuesta
print(f"URL: {url}")
print(f"Código de estado: {respuesta.status_code}")  # 200 = éxito
print(f"Tipo de contenido: {respuesta.headers.get('Content-Type', 'N/A')}")
print(f"Tamaño: {len(respuesta.text):,} caracteres")

### Códigos de estado HTTP

| Código | Significado |
|--------|------------|
| **200** | ✅ Éxito — la página se cargó bien |
| **301/302** | ↩️ Redirección — la página se movió |
| **403** | 🚫 Prohibido — no tienes acceso |
| **404** | ❌ No encontrado — la página no existe |
| **429** | ⏱️ Demasiadas peticiones — te bloquearon temporalmente |
| **500** | 💥 Error del servidor |


In [ ]:
# Ver el HTML crudo (primeros 500 caracteres)
print(respuesta.text[:500])

### 2.2 Headers: identificarte como navegador

Algunos sitios bloquean peticiones que no parecen venir de un navegador real. Podemos enviar headers para identificarnos:


In [ ]:
# Buena práctica: enviar un User-Agent
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

respuesta = requests.get("https://quotes.toscrape.com/", headers=headers)
print(f"Status: {respuesta.status_code}")

---

## 3. Parsear HTML con BeautifulSoup

BeautifulSoup convierte el HTML en una estructura que puedes navegar fácilmente.

### 3.1 Crear el soup


In [ ]:
# Convertir HTML a BeautifulSoup
url = "https://quotes.toscrape.com/"
respuesta = requests.get(url)
soup = BeautifulSoup(respuesta.text, "lxml")

# El título de la página
print(f"Título: {soup.title.text}")

### 3.2 Buscar elementos

BeautifulSoup tiene dos métodos principales para buscar:

| Método | Qué devuelve |
|--------|-------------|
| `find()` | El **primer** elemento que coincida |
| `find_all()` | **Todos** los elementos que coincidan (lista) |

Puedes buscar por:
- **Etiqueta:** `soup.find("h1")` — busca `<h1>`
- **Clase CSS:** `soup.find("div", class_="quote")` — busca `<div class="quote">`
- **ID:** `soup.find("div", id="header")` — busca `<div id="header">`
- **Selector CSS:** `soup.select("div.quote span.text")` — más flexible


In [ ]:
# Encontrar el primer elemento de un tipo
primer_h1 = soup.find("h1")
print(f"Primer H1: {primer_h1.text}")

# Encontrar por clase CSS
primera_cita = soup.find("div", class_="quote")
print(f"\nPrimera cita completa (HTML):")
print(primera_cita.prettify()[:300])

In [ ]:
# Encontrar TODOS los elementos
todas_citas = soup.find_all("div", class_="quote")
print(f"Total de citas encontradas: {len(todas_citas)}")

# Extraer texto de la primera cita
cita = todas_citas[0]
texto = cita.find("span", class_="text").text
autor = cita.find("small", class_="author").text
tags = [tag.text for tag in cita.find_all("a", class_="tag")]

print(f"\nTexto: {texto}")
print(f"Autor: {autor}")
print(f"Tags: {tags}")

### 3.3 Extraer datos estructurados


In [ ]:
# Extraer TODAS las citas de la primera página
url = "https://quotes.toscrape.com/"
respuesta = requests.get(url)
soup = BeautifulSoup(respuesta.text, "lxml")

citas = []
for div in soup.find_all("div", class_="quote"):
    texto = div.find("span", class_="text").text
    autor = div.find("small", class_="author").text
    tags = [tag.text for tag in div.find_all("a", class_="tag")]

    citas.append({
        "texto": texto,
        "autor": autor,
        "tags": ", ".join(tags),
    })

# Convertir a DataFrame
df_citas = pd.DataFrame(citas)
print(f"Citas extraídas: {len(df_citas)}")
df_citas

---

## 4. Navegación avanzada y selectores CSS

### 4.1 Selectores CSS con select()

`select()` usa la misma sintaxis que CSS — si sabes CSS, es muy poderoso:


In [ ]:
# Selectores CSS
url = "https://quotes.toscrape.com/"
respuesta = requests.get(url)
soup = BeautifulSoup(respuesta.text, "lxml")

# Selector por clase
textos = soup.select("span.text")
print(f"Citas encontradas con select: {len(textos)}")
print(f"Primera: {textos[0].text[:60]}...")

# Selector anidado: links dentro de divs con clase quote
tags_links = soup.select("div.quote a.tag")
print(f"\nTags encontrados: {len(tags_links)}")
tags_unicos = list(set(tag.text for tag in tags_links))
print(f"Tags únicos: {sorted(tags_unicos)}")

### 4.2 Extraer atributos (href, src, etc.)


In [ ]:
# Extraer todos los links de la página
links = soup.find_all("a")
print(f"Links encontrados: {len(links)}\n")

for link in links[:10]:
    texto = link.text.strip()
    href = link.get("href", "Sin href")
    if texto:
        print(f"  [{texto}] → {href}")

---

## 5. Scraping de múltiples páginas (paginación)

La mayoría de sitios dividen sus datos en varias páginas. Necesitamos navegar entre ellas.


In [ ]:
# Extraer citas de TODAS las páginas
todas_citas = []
pagina = 1

while True:
    url = f"https://quotes.toscrape.com/page/{pagina}/"
    respuesta = requests.get(url)
    soup = BeautifulSoup(respuesta.text, "lxml")

    citas_pagina = soup.find_all("div", class_="quote")

    # Si no hay citas, terminamos
    if not citas_pagina:
        break

    for div in citas_pagina:
        texto = div.find("span", class_="text").text
        autor = div.find("small", class_="author").text
        tags = [tag.text for tag in div.find_all("a", class_="tag")]

        todas_citas.append({
            "texto": texto,
            "autor": autor,
            "tags": ", ".join(tags),
            "pagina": pagina,
        })

    print(f"  Página {pagina}: {len(citas_pagina)} citas extraídas")
    pagina += 1

    # ⚠️ IMPORTANTE: esperar entre peticiones para no sobrecargar el servidor
    time.sleep(1)

df_todas = pd.DataFrame(todas_citas)
print(f"\n✅ Total: {len(df_todas)} citas de {pagina - 1} páginas")
df_todas.head()

In [ ]:
# Análisis rápido de las citas
print(f"Total de citas: {len(df_todas)}")
print(f"Autores únicos: {df_todas['autor'].nunique()}")

print(f"\n📊 Top 5 autores con más citas:")
print(df_todas["autor"].value_counts().head())

# Todos los tags
todos_tags = []
for tags_str in df_todas["tags"]:
    todos_tags.extend(tags_str.split(", "))

tags_series = pd.Series(todos_tags)
print(f"\n🏷️ Top 10 tags:")
print(tags_series.value_counts().head(10))

---

## 6. Extraer tablas HTML

Las tablas HTML son de lo más común que querrás extraer. Pandas tiene un atajo increíble para esto.

### 6.1 Con Pandas (la forma fácil)


In [ ]:
# Pandas puede leer tablas HTML directamente — ¡una línea!
url = "https://www.worldometers.info/world-population/population-by-country/"

# read_html devuelve una LISTA de todas las tablas de la página
tablas = pd.read_html(url)
print(f"Tablas encontradas: {len(tablas)}")

# La primera tabla suele ser la principal
df_paises = tablas[0]
print(f"\nColumnas: {list(df_paises.columns)}")
print(f"Filas: {len(df_paises)}")
df_paises.head(10)

In [ ]:
# Filtrar: países de América Latina (algunos ejemplos)
latam = ["Mexico", "Brazil", "Argentina", "Colombia", "Chile", "Peru",
         "Ecuador", "Guatemala", "Cuba", "Bolivia", "Honduras",
         "Paraguay", "El Salvador", "Nicaragua", "Costa Rica", "Panama", "Uruguay"]

# Buscar la columna que contiene el nombre del país
col_pais = df_paises.columns[1]  # Generalmente la segunda columna
col_poblacion = df_paises.columns[2]  # Generalmente la tercera

df_latam = df_paises[df_paises[col_pais].isin(latam)].copy()
print(f"Países de LATAM encontrados: {len(df_latam)}")
df_latam[[col_pais, col_poblacion]].head(15)

### 6.2 Con BeautifulSoup (más control)


In [ ]:
# Extraer tabla manualmente con BeautifulSoup
url = "https://quotes.toscrape.com/tableful/"

try:
    respuesta = requests.get(url, timeout=5)
    soup = BeautifulSoup(respuesta.text, "lxml")

    # Buscar la tabla
    tabla = soup.find("table")
    if tabla:
        # Extraer encabezados
        encabezados = [th.text.strip() for th in tabla.find_all("th")]
        print(f"Encabezados: {encabezados}")

        # Extraer filas
        filas = []
        for tr in tabla.find_all("tr")[1:]:  # Saltar el encabezado
            fila = [td.text.strip() for td in tr.find_all("td")]
            if fila:
                filas.append(fila)

        df_tabla = pd.DataFrame(filas, columns=encabezados if encabezados else None)
        print(f"Filas extraídas: {len(df_tabla)}")
        print(df_tabla.head())
    else:
        print("No se encontró tabla en esta página")
        print("(Esto es normal — no todas las páginas tienen tablas)")
except Exception as e:
    print(f"Nota: {e}")
    print("Esto es esperado si la URL no tiene tabla. Usemos el ejemplo de worldometers arriba.")

---

## 7. Ejemplo práctico: Extraer datos de libros

Vamos a hacer scraping de un sitio diseñado para practicar: [books.toscrape.com](http://books.toscrape.com/)


In [ ]:
# Extraer información de libros
url = "http://books.toscrape.com/"
respuesta = requests.get(url)
soup = BeautifulSoup(respuesta.text, "lxml")

# Cada libro está en un <article class="product_pod">
libros = []
for articulo in soup.find_all("article", class_="product_pod"):
    # Título
    titulo = articulo.find("h3").find("a")["title"]

    # Precio
    precio_texto = articulo.find("p", class_="price_color").text
    precio = float(precio_texto.replace("£", "").replace("Â", ""))

    # Rating (está como clase CSS: "star-rating Three" etc.)
    rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
    rating_class = articulo.find("p", class_="star-rating")["class"][1]
    rating = rating_map.get(rating_class, 0)

    # Disponibilidad
    disponible = "In stock" in articulo.find("p", class_="instock").text

    # URL del libro
    link = articulo.find("h3").find("a")["href"]

    libros.append({
        "titulo": titulo,
        "precio_gbp": precio,
        "rating": rating,
        "disponible": disponible,
        "url": f"http://books.toscrape.com/{link}",
    })

df_libros = pd.DataFrame(libros)
print(f"✅ {len(df_libros)} libros extraídos de la primera página")
df_libros

In [ ]:
# Análisis rápido
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Distribución de precios
axes[0].hist(df_libros["precio_gbp"], bins=10, color="#2E86AB", edgecolor="white")
axes[0].set_title("Distribución de Precios", fontweight="bold")
axes[0].set_xlabel("Precio (£)")

# Distribución de ratings
df_libros["rating"].value_counts().sort_index().plot(kind="bar", ax=axes[1], color="#A23B72")
axes[1].set_title("Distribución de Ratings", fontweight="bold")
axes[1].set_xlabel("Estrellas")
axes[1].set_ylabel("Cantidad")

# Precio promedio por rating
df_libros.groupby("rating")["precio_gbp"].mean().plot(kind="bar", ax=axes[2], color="#F18F01")
axes[2].set_title("Precio Promedio por Rating", fontweight="bold")
axes[2].set_xlabel("Estrellas")
axes[2].set_ylabel("Precio (£)")

for ax in axes:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()

print(f"Precio promedio: £{df_libros['precio_gbp'].mean():.2f}")
print(f"Libro más caro: {df_libros.loc[df_libros['precio_gbp'].idxmax(), 'titulo']} (£{df_libros['precio_gbp'].max()})")
print(f"Libro más barato: {df_libros.loc[df_libros['precio_gbp'].idxmin(), 'titulo']} (£{df_libros['precio_gbp'].min()})")

In [ ]:
# Extraer libros de las primeras 5 páginas
todos_libros = []

for pagina in range(1, 6):  # Páginas 1 a 5
    url = f"http://books.toscrape.com/catalogue/page-{pagina}.html"
    respuesta = requests.get(url)

    if respuesta.status_code != 200:
        print(f"  Página {pagina}: Error {respuesta.status_code}")
        break

    soup = BeautifulSoup(respuesta.text, "lxml")
    rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

    for art in soup.find_all("article", class_="product_pod"):
        titulo = art.find("h3").find("a")["title"]
        precio = float(art.find("p", class_="price_color").text.replace("£", "").replace("Â", ""))
        rating = rating_map.get(art.find("p", class_="star-rating")["class"][1], 0)

        todos_libros.append({
            "titulo": titulo,
            "precio_gbp": precio,
            "rating": rating,
            "pagina": pagina,
        })

    print(f"  Página {pagina}: {len(soup.find_all('article', class_='product_pod'))} libros")
    time.sleep(0.5)  # Respetar el servidor

df_todos = pd.DataFrame(todos_libros)
print(f"\n✅ Total: {len(df_todos)} libros extraídos")
print(f"Precio promedio: £{df_todos['precio_gbp'].mean():.2f}")
print(f"Rating promedio: {df_todos['rating'].mean():.1f} estrellas")

---

## 8. Buenas prácticas

### 8.1 Revisar robots.txt

Siempre revisa el archivo `robots.txt` de un sitio antes de hacer scraping:


In [ ]:
# Verificar robots.txt
url = "http://books.toscrape.com/robots.txt"
respuesta = requests.get(url)

if respuesta.status_code == 200:
    print(f"robots.txt de books.toscrape.com:")
    print(respuesta.text)
else:
    print(f"No hay robots.txt (status: {respuesta.status_code})")
    print("Esto significa que no hay restricciones explícitas")

### 8.2 Función de scraping robusta

Una función que maneja errores, reintentos y límites:


In [ ]:
def obtener_pagina(url: str, max_reintentos: int = 3, espera: float = 1.0) -> BeautifulSoup | None:
    """
    Descarga y parsea una página web de forma robusta.

    Args:
        url: URL de la página a descargar.
        max_reintentos: Número máximo de intentos si falla.
        espera: Segundos a esperar entre reintentos.

    Returns:
        Objeto BeautifulSoup o None si falló.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (compatible; PythonBot/1.0; +https://culiacan.ai)"
    }

    for intento in range(max_reintentos):
        try:
            respuesta = requests.get(url, headers=headers, timeout=10)

            if respuesta.status_code == 200:
                return BeautifulSoup(respuesta.text, "lxml")
            elif respuesta.status_code == 429:
                print(f"⏱️ Demasiadas peticiones, esperando {espera * (intento + 1)}s...")
                time.sleep(espera * (intento + 1))
            else:
                print(f"⚠️ Status {respuesta.status_code} para {url}")
                return None

        except requests.exceptions.Timeout:
            print(f"⏱️ Timeout en intento {intento + 1}")
            time.sleep(espera)
        except requests.exceptions.ConnectionError:
            print(f"🔌 Error de conexión en intento {intento + 1}")
            time.sleep(espera)

    print(f"❌ No se pudo acceder a {url} después de {max_reintentos} intentos")
    return None

# Probar la función
soup = obtener_pagina("https://quotes.toscrape.com/")
if soup:
    print(f"✅ Página descargada: {soup.title.text}")

### 8.3 Checklist de buenas prácticas

Antes de hacer scraping, siempre:

1. ✅ **Revisa si hay una API** — es mejor que scraping
2. ✅ **Lee robots.txt** — `sitio.com/robots.txt`
3. ✅ **Lee los términos de servicio** del sitio
4. ✅ **Agrega pausas** entre peticiones (`time.sleep()`)
5. ✅ **Usa User-Agent** apropiado
6. ✅ **Maneja errores** con try/except
7. ✅ **No sobrecargues** el servidor (máximo 1 petición por segundo)
8. ✅ **Guarda los datos** para no repetir el scraping innecesariamente
9. ✅ **Cita la fuente** cuando uses los datos


---

## 9. Guardar resultados


In [ ]:
import os
os.makedirs("datos", exist_ok=True)

# Guardar citas
df_todas.to_csv("datos/citas_scraping.csv", index=False)
print(f"✅ citas_scraping.csv: {len(df_todas)} citas")

# Guardar libros
df_todos.to_csv("datos/libros_scraping.csv", index=False)
print(f"✅ libros_scraping.csv: {len(df_todos)} libros")

# También en Excel
with pd.ExcelWriter("datos/scraping_resultados.xlsx") as writer:
    df_todas.to_excel(writer, sheet_name="Citas", index=False)
    df_todos.to_excel(writer, sheet_name="Libros", index=False)

print("✅ scraping_resultados.xlsx (2 hojas)")

---

## 10. 🏆 Mini Proyecto: Scraper de noticias de Wikipedia

Vamos a extraer la tabla de los países más poblados del mundo desde Wikipedia:


In [ ]:
# 🏆 Mini Proyecto: Datos de países desde Wikipedia

# Wikipedia permite scraping y sus tablas son fáciles de extraer con Pandas
url = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population"

print("📥 Descargando datos de Wikipedia...")
tablas = pd.read_html(url)
print(f"Tablas encontradas: {len(tablas)}")

# La tabla principal suele ser la más grande
df = max(tablas, key=len)
print(f"Tabla principal: {len(df)} filas, {len(df.columns)} columnas")
print(f"Columnas: {list(df.columns)}")
df.head(10)

In [ ]:
# Limpiar y analizar
# Nota: las columnas pueden variar, ajustamos según lo que encontremos

# Renombrar columnas para facilitar el trabajo
df_clean = df.copy()
df_clean.columns = [str(c).strip() for c in df_clean.columns]

print("Columnas disponibles:")
for i, col in enumerate(df_clean.columns):
    print(f"  [{i}] {col}")

# Mostrar las primeras filas para entender la estructura
print("\nPrimeras filas:")
df_clean.head()

In [ ]:
# Visualización rápida: Top 15 países más poblados
import matplotlib.pyplot as plt

# Intentar identificar las columnas correctas
# Generalmente: columna 1 = país, columna 2 = población
col_pais = df_clean.columns[1]
col_poblacion = df_clean.columns[2]

# Tomar top 15
top15 = df_clean.head(15).copy()

fig, ax = plt.subplots(figsize=(10, 8))

# Limpiar datos de población (quitar comas, convertir a número)
try:
    top15[col_poblacion] = pd.to_numeric(
        top15[col_poblacion].astype(str).str.replace(",", "").str.replace(" ", ""),
        errors="coerce"
    )

    top15_sorted = top15.sort_values(col_poblacion, ascending=True)

    ax.barh(top15_sorted[col_pais].astype(str), top15_sorted[col_poblacion],
            color="#2E86AB", edgecolor="white")
    ax.set_title("Top 15 Países Más Poblados del Mundo", fontsize=14, fontweight="bold")
    ax.set_xlabel("Población")
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"{x/1e6:.0f}M"))
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    plt.tight_layout()
    plt.show()

    # Guardar resultado limpio
    top15.to_csv("datos/top15_paises_poblacion.csv", index=False)
    print("\n✅ top15_paises_poblacion.csv guardado")
except Exception as e:
    print(f"Nota: La estructura de Wikipedia puede variar. Error: {e}")
    print("Intenta ajustar las columnas manualmente basándote en el output anterior")

---

## 🔥 Retos

1. **Scraper de libros completo:** Extrae TODAS las páginas (50) de books.toscrape.com. Analiza: distribución de precios por categoría, rating promedio, y los 10 libros más caros.

2. **Scraper de autores:** En quotes.toscrape.com, cada autor tiene una página con su biografía (`/author/nombre`). Extrae el nombre, fecha de nacimiento, lugar y descripción de los 10 primeros autores.

3. **Wikipedia LATAM:** Extrae de Wikipedia la tabla de población de países de América Latina. Limpia los datos, calcula el total, y genera una gráfica de barras con el top 10.


In [ ]:
# Reto 1: Scraper de libros completo
# Tu código aquí 👇


In [ ]:
# Reto 2: Scraper de autores
# Tu código aquí 👇


In [ ]:
# Reto 3: Wikipedia LATAM
# Tu código aquí 👇


---

## 📋 Resumen

### Proceso de web scraping

| Paso | Herramienta | Código |
|------|------------|--------|
| 1. Descargar HTML | `requests` | `requests.get(url)` |
| 2. Parsear HTML | `BeautifulSoup` | `BeautifulSoup(html, "lxml")` |
| 3. Buscar elementos | BS4 | `soup.find()`, `soup.find_all()`, `soup.select()` |
| 4. Extraer datos | BS4 | `.text`, `.get("href")`, `["class"]` |
| 5. Tablas directo | `Pandas` | `pd.read_html(url)` |
| 6. Guardar | `Pandas` | `df.to_csv()`, `df.to_excel()` |

### Métodos clave de BeautifulSoup

| Método | Ejemplo | Qué hace |
|--------|---------|---------|
| `find()` | `soup.find("div", class_="quote")` | Primer elemento |
| `find_all()` | `soup.find_all("a")` | Todos los elementos |
| `select()` | `soup.select("div.quote span.text")` | Selector CSS |
| `.text` | `elemento.text` | Texto del elemento |
| `.get()` | `link.get("href")` | Valor de un atributo |

---

## ⏭️ ¿Qué sigue?

En el siguiente notebook aprenderás sobre **APIs y JSON** — cómo consumir datos de servicios web de forma estructurada y profesional.

👉 [10 — APIs y JSON](10_APIs_y_JSON.ipynb)

---

<p align="center">
  Hecho con ❤️ por <a href="https://culiacan.ai">Culiacan.AI</a> — Culiacán reconocida en el mundo por su talento y emprendimiento en Inteligencia Artificial
</p>
